In [ ]:
# display plots inline
%matplotlib notebook

# imports
import os
import numpy as np
import pandas as pd
import pymc3 as pm
from bambi import Model, Prior
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import pymc3_utils as pmu

# suppress system warnings for legibility
import warnings
warnings.filterwarnings('ignore')

# resize plots to fit labels inside bounding box
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

# MPI color scheme
sns.set(style='white', palette='Set2')

# Digit span
## Loading data

In [ ]:
df_span = pd.read_csv('data/digit_span.tsv', sep='\t')
df_span = pd.melt(df_span, id_vars=['pp'], value_vars=['fds', 'bds'], var_name='task', value_name='span')
df_reading = pd.read_csv('data/tamil_reading.tsv', sep='\t')
df_span = df_span.merge(df_reading, left_on='pp', right_on='pp')

display(df_span.head())

## Data mangling and plotting

In [ ]:
# standardize variables
df_span['span_z'] = pmu.standardize(df_span['span'])
df_span['reading_z'] = pmu.standardize(df_span['word'])
df_span['task_binary'] = pd.get_dummies(df_span['task'])['fds']
df_span['task_z'] = pmu.standardize(df_span['task_binary'])

display(df_span.head())

## Modeling

First we will set some parameters for the regression procedure, as outlined in the paper.

In [ ]:
# default model params
defaults = {
    'samples': 5000,
    'tune': 2500,
    'chains': 4,
    'init': 'advi+adapt_diag',
    'priors': {'fixed': 'narrow', 'random': 'narrow'},
}

We will use the only model that makes sense in the context of our multilevel model setup: A task model, an additional predictor for task type (fds versus bds) and by-participant intercepts.

In [ ]:
model_span_task = Model(df_span)
model_span_task.fit('span_z ~ task_z',
                    random=['1|pp'],
                    **defaults)

In [ ]:
display(pmu.summary(model_span_task.backend.trace).round(2))

\\(\hat{r}\\) values look good, as does the number of effective samples for the by-participant intercepts. For the overall intercept we have more than 8K effective samples, for the span variance we have more than 5.5K.

## Plotting selected model traces

As an additional check, we will plot the posterior traces for the model. Ideally these look unimodal and roughly normal. The plots on the righthand side should look like fuzzy caterpillars.

In [ ]:
g_traces = pm.traceplot(model_span_task.backend.trace)
plt.savefig('figures/digit_span_model_traces.png', dpi=600)

## Extracting participant intercept modes

We will now extract and store the posterior modes of the participant intercepts so we can use them to model reading scores.

In [ ]:
pps = df_span['pp'].unique()
pp_nums = [f'1|pp__{i}' for i in range(len(pps))]
df_intercepts = pmu.summary(model_span_task.backend.trace).loc[pp_nums]
df_intercepts['pp'] = np.sort(pps)

display(df_intercepts.head().round(2))

In [ ]:
df_uncorrected = df_span.groupby('pp', as_index=False).mean().rename(columns={'span': 'raw_span_mean'})
df_intercepts = df_intercepts[['pp', 'mode']].rename(columns={'mode': 'span_intercept'})
df_intercepts = df_intercepts.merge(df_uncorrected[['pp', 'reading_z', 'raw_span_mean']],
                                    left_on='pp', right_on='pp').reset_index()

display(df_intercepts.head().round(2))

In [ ]:
# and write to file
df_intercepts.to_csv('data/span_intercepts.tsv', sep='\t')

Before closing this notebook, we will take a quick look at the correlations between working memory and reading score.

In [ ]:
display(df_intercepts[['raw_span_mean', 'span_intercept', 'reading_z']].corr().round(2))